Задание<br>
Берем отызывы за лето (из архива с материалами или предыдущего занятия)
1. Учим conv сеть для классификации
2. Рассмотреть 2-а варианта сеточек
2.1 Инициализировать tf.keras.layers.Embedding предобученными векторами взять к примеру с https://rusvectores.org/ru/
2.2 Инициализировать слой tf.keras.layers.Embedding по умолчанию (ну то есть вам ничего не делать с весами)

Сравнить две архитектуры с предобученными весами и когда tf.keras.layers.Embedding обучается сразу со всей сеточкой, что получилось лучше

In [65]:
import pandas as pd

df = pd.read_excel('отзывы за лето.xls')
df.head()

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


### Предобработка

In [66]:
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

In [67]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df['text'] = df['Content'].apply(preprocess_text)
df.head()

,Rating,Content,Date,text
0,5,It just works!,2017-08-14,it just works
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14,целое удобноной приложениеиз минус хотеть боль...
2,5,Отлично все,2017-08-14,отлично
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14,зависать 1 работа антивирус ранее пользоваться...
4,5,"Очень удобно, работает быстро.",2017-08-14,удобно работать быстро


In [68]:
train_corpus = " ".join(df["text"])
train_corpus = train_corpus.lower()

In [69]:
# train_corpus

In [70]:
import nltk
from nltk.tokenize import word_tokenize
# nltk.download("punkt")

tokens = word_tokenize(train_corpus)

Отфильтруем данные

и соберём в корпус N наиболее частых токенов

In [71]:
tokens_filtered = [word for word in tokens if word.isalnum()]

In [72]:
max_words = 200

from nltk.probability import FreqDist
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [73]:
tokens_filtered_top[:10]

['приложение',
 'удобно',
 'работать',
 'удобный',
 'отлично',
 'нравиться',
 'хороший',
 'отличный',
 'телефон',
 'супер']

In [74]:
# from sklearn.model_selection import train_test_split

# df_train, df_test = train_test_split(df, test_size=0.2,  random_state=42)
# df_train, df_val  = train_test_split(df_train, test_size=0.2, random_state=42)

In [75]:
import numpy as np

max_len = 40

vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

x_train = np.asarray([text_to_sequence(text, max_len) for text in df["text"]], dtype=np.int32)

# x_train = np.asarray([text_to_sequence(text, max_len) for text in df_train["text"]], dtype=np.int32)
# x_test  = np.asarray([text_to_sequence(text, max_len) for text in df_test["text"]], dtype=np.int32)
# x_val   = np.asarray([text_to_sequence(text, max_len) for text in df_val["text"]], dtype=np.int32)

In [76]:
print(x_train.shape)
# print(x_test.shape)
# print(x_val.shape)

(20659, 40)


### Keras model

In [77]:
import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D 
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.objectives import categorical_crossentropy
from keras.callbacks import EarlyStopping  

In [78]:
num_classes = 6

y_train = keras.utils.to_categorical(df["Rating"], num_classes)
# y_test  = keras.utils.to_categorical(df_test["Rating"], num_classes)
# y_val   = keras.utils.to_categorical(df_val["Rating"], num_classes)

In [79]:
# max_words = 1000
# max_len = 40

# print_batch_n = 100 

model = Sequential()

model.add(Embedding(input_dim=max_words, 
                    output_dim=200, 
                    input_length=max_len,
#                     weights=[model_w2v.trainables.syn1neg[:max_words]]
         ))
          
model.add(Conv1D(128, 3, activation='relu'))
model.add(Conv1D(256, 3, activation='relu'))
model.add(Conv1D(200, 3, activation='relu'))
model.add(GlobalMaxPool1D())
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# model.summary()

In [80]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy']
#               metrics=['AUC']
             )

In [81]:
print(x_train.shape)
print(y_train.shape)

# print(x_val.shape)
# print(y_val.shape)

(20659, 40)
(20659, 6)


In [82]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  

epochs = 15
batch_size = 512

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.3,
                    callbacks=[tensorboard, early_stopping]
                   )

Epoch 1/15
29/29 [==============================] - 10s 310ms/step - loss: 1.3410 - accuracy: 0.7289 - val_loss: 1.0193 - val_accuracy: 0.6526
Epoch 2/15
29/29 [==============================] - 6s 213ms/step - loss: 0.8038 - accuracy: 0.7444 - val_loss: 0.7799 - val_accuracy: 0.7430
Epoch 3/15
29/29 [==============================] - 6s 212ms/step - loss: 0.6593 - accuracy: 0.7676 - val_loss: 0.7583 - val_accuracy: 0.7462
Epoch 4/15
29/29 [==============================] - 6s 212ms/step - loss: 0.6561 - accuracy: 0.7696 - val_loss: 0.7549 - val_accuracy: 0.7425
Epoch 5/15
29/29 [==============================] - 6s 211ms/step - loss: 0.6326 - accuracy: 0.7805 - val_loss: 0.7557 - val_accuracy: 0.7456


# LogisticRegression

In [92]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score 

In [93]:
vect = TfidfVectorizer(ngram_range=(1, 2), analyzer='word', lowercase=False)

In [94]:
train_ft = vect.fit_transform(df['text'])
# valid_ft = vect.transform(df_val['text'])

In [95]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
                    train_ft, 
                    df['Rating'].to_numpy(), 
                    test_size=0.33, 
                    random_state=42)

In [96]:
lgr = LogisticRegression()

In [97]:
lgr.fit(X_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [98]:
y_pred = lgr.predict(X_test)

In [99]:
accuracy_score(y_test, y_pred)

0.7606336168964506

# Word2Vec

In [83]:
from gensim.models import Word2Vec

In [84]:
modelW2V = Word2Vec(sentences=df['text'].apply(str.split), 
                    size=100, 
                    window=5, 
                    min_count=5, 
                    workers=8)

In [85]:
# modelW2V.wv.vocab

In [86]:
sentences = [[i for i in document.split()] for document in df['text'].tolist()]

In [87]:
model_w2v = Word2Vec(sentences=sentences, 
                     size=200, 
                     window=5,
                     min_count=6, 
                     sg=1, 
                     hs=0, 
                     negative=10, 
                     workers=32, 
                     seed=34)

model_w2v.train(sentences, total_examples=len(sentences), epochs=20)

(1314772, 2062320)

In [88]:
# model_w2v.most_similar(positive=['минус'])
# aaa = [model_w2v.trainables.syn1neg[:max_words]]
# print(aaa)

In [89]:
# max_words = 1000
# max_len = 40

# print_batch_n = 100 

model = Sequential()

model.add(Embedding(input_dim=max_words, 
                    output_dim=200, 
                    input_length=max_len,
                    weights=[model_w2v.trainables.syn1neg[:max_words]]
         ))
          
model.add(Conv1D(128, 3, activation='relu'))
model.add(Conv1D(256, 3, activation='relu'))
model.add(Conv1D(200, 3, activation='relu'))
model.add(GlobalMaxPool1D())
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# model.summary()

In [90]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy']
#               metrics=['AUC']
             )

In [91]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  

# epochs = 15
# batch_size = 512

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.3,
                    callbacks=[tensorboard, early_stopping]
                   )

Epoch 1/15
29/29 [==============================] - 8s 260ms/step - loss: 1.1202 - accuracy: 0.6584 - val_loss: 0.9389 - val_accuracy: 0.6615
Epoch 2/15
29/29 [==============================] - 6s 214ms/step - loss: 0.7627 - accuracy: 0.7490 - val_loss: 0.7887 - val_accuracy: 0.7349
Epoch 3/15
29/29 [==============================] - 6s 214ms/step - loss: 0.6777 - accuracy: 0.7655 - val_loss: 0.7660 - val_accuracy: 0.7439
Epoch 4/15
29/29 [==============================] - 6s 223ms/step - loss: 0.6677 - accuracy: 0.7695 - val_loss: 0.7587 - val_accuracy: 0.7462
Epoch 5/15
29/29 [==============================] - 6s 220ms/step - loss: 0.6511 - accuracy: 0.7732 - val_loss: 0.7663 - val_accuracy: 0.7460
